# Ch. 5: Symbolic Checking - Beyond Testing

Unit tests (Ch.3) and property-based tests (Ch.4) give high confidence, but they still sample the input space. In this chapter we climb the next rung on the “ladder of rigor”: **symbolic/bounded checking.** We’ll:

 - Encode **contracts** (preconditions, postconditions, invariants) as executable checks.
 - Use **bounded checking** to exhaustively explore small input domains.
 - Use **CrossHair** to do symbolic execution on pure-Python functions and automatically find counterexamples (or prove safety within a bound).

This isn’t full-blown theorem proving; it’s the pragmatic middle: fast feedback and stronger guarantees for core routines.

[TODO]

> Testing can be used to show the presence of bugs, but never to show their absence. - Edsger W. Dijkstra

[TODO]

**Small Scope Hypothesis:** A high proportion of bugs can be found by testing a program for all test inputs within some small scope.
[todo: add the coverage graph]


## CrossHair

CrossHair tries to prove or refute your `assert`s by exploring the possible execution paths of your code with a symbolic execution engine. It's perfect for small functions. Hence, another
reason for having modular code with functions that have single responsibility and clear APIs.

Just like the `pytest` framework, CrossHair is used as a command-line tool, so we'll mimic
the workflow we adopted for `pytest`:

1. Using the `%%writefile` magic command, we will save our test codes to Python files.
2. We will then run the tests using the command line command `crosshair`. Recall, in Jupyter notebooks, we can run such shell commands by prefixing them with `!`.

In [22]:
%%writefile check_div.py

def div(x: float, y: float) -> float:
    assert y != 0           # P    (precondition)
    res = x / y             # code (implementation)
    assert res * y == x     # Q    (postcondition)
    return res

Overwriting check_div.py


In [23]:
!crosshair check --analysis_kind=asserts check_div.py

/Users/altuntas/r3sw/notebooks/check_div.py:5: error: AssertionError:  when calling div(0.0, float("inf"))


Based on the counterexample returned by CrossHair, let's refine our precondition to include checks for infinity:

In [43]:
%%writefile check_div.py

def div(x: float, y: float) -> float:
    assert y != 0                       # P    (precondition)
    assert float('inf') not in (x, y)   # P cont'd: check for infinity
    res = x / y                         # code (implementation)
    assert res * y == x                 # Q    (postcondition)
    return res

Overwriting check_div.py


In [44]:
!crosshair check --analysis_kind=asserts check_div.py

With this refined precondition, CrossHair no longer reports any counterexamples.
But what about the edge cases we discovered earlier, such as the 7/25 scenario?
CrossHair fails to detect these because, as a symbolic execution tool, it reasons primarily over real numbers, not actual floating-point values.

This distinction matters. Real arithmetic is exact and free of rounding, while floating-point arithmetic introduces finite precision, rounding errors, and special values like NaN and inf.
As a result, subtle bugs that arise from precision loss or rounding behavior, such as the slight inaccuracy of 7/25—can slip past CrossHair’s analysis.

Even with this limitation, CrossHair remains a powerful tool for uncovering a wide range of other issues: missing or overly weak preconditions, invalid assumptions about inputs, computational bugs, and logical errors in algorithms.
These are often the most challenging and costly bugs to identify through traditional testing alone.

In fact, this limitation can also be seen as an opportunity. By reasoning in terms of real arithmetic, CrossHair allows us to abstract away from low-level floating-point inaccuracies and focus instead on the mathematical and computational correctness of our scientific applications.
This higher-level perspective helps us verify the intended behavior of our algorithms and modeling machinery without getting bogged down by the intricacies of floating-point implementation details.

Once the core logic is proven sound in this idealized setting, we can then address floating-point concerns separately by using complementary techniques like unit testing or 
property-based testing.

In this way, CrossHair serves as a valuable partner: it ensures that our scientific software is conceptually correct, while leaving room for other tools and strategies to handle the unavoidable complexities of real-world floating-point computation.

## Back to the 1-D Heat Equation Solver

Now let's use CrossHair to test the core functions of our heat equation solver.

[TODO}]

...



## What we just did

 - We promoted core properties (telescoping, symmetry, linearity) into machine-checked contracts.
 - We combined two tactics:
   - Bounded checking (exhaustive on tiny domains).
   - Symbolic execution with CrossHair (auto-finds counterexamples).
 - This gives deeper, earlier confidence in the computational heart of the solver—before large simulations run.

## Looking Ahead

You now have the full ladder:

 - Ch.1: working baseline (code & fix)
 - Ch.2: specs & structure
 - Ch.3: unit tests
 - Ch.4: property-based tests
 - Ch.5: symbolic & bounded checking

In practice, you’d mix these: unit/property tests run all the time including within automated CI/CD testing pipelines. Symbolic/bounded checks run on small kernels when stronger guarantees are warranted. Together, they shift scientific software toward explicit reasoning and trustworthy results, without heavy ceremony.